Tout d'abord, récupérer les fichiers de séquences génomiques fasta.
Utiliser le script ci-dessous afin d'extraire les séquences chromosomiques de chaque génome dans des fichiers fasta séparés
Modifier l'input_file ainsi que les chromosomes_ids pour chaque génome différent.
Si besoin, installer Biopython

Activer un environnement conda : 

In [ ]:
conda activate environpro

Si utilisation du serveur ifb, aller dans le chemin de votre compte et charger les modules correspondants :

In [ ]:
cd /shared/home/hnguyen97/ondemand/data/sys/dashboard/batch_connect/sys/jupyter/core/projet_pangenome
module load pggb


(Si besoin :)

In [ ]:
conda install biopython

Utiliser le fichier python extraction_chromosome.py :

In [ ]:

from Bio import SeqIO
import os
import re

repertoire = "Projet_Pangenome/genomes"
input_list = []

for nom_fichier in os.listdir(repertoire):
    chemin_complet = os.path.join(repertoire, nom_fichier)
    input_list.append(chemin_complet)

pattern = re.compile(r"(.+)\/genomes\/(.+)\.genome\.fasta")

def extraction_chromosome():
    for file in input_list:
        print(file)
        # Dossier de sortie
        match = pattern.match(file)
        if match:
            print(match.group(2))                 
            output_folder = f"Projet_Pangenome/Chromosomes_{match.group(2)}"
            os.makedirs(output_folder, exist_ok=True)
            # Lecture et extraction des séquences
            for record in SeqIO.parse(file, "fasta"):
                    # Construire un fichier de sortie basé sur l'identifiant
                    output_file = os.path.join(output_folder,f"{record.id}.fasta")
                    with open(output_file, "w") as f:
                        SeqIO.write(record, f, "fasta")
                    print(f"Séquence {record.id} sauvegardée dans {output_file}")
    print("Extraction terminée.")

def main():
    extraction_chromosome()

if __name__ == '__main__':
    main()

**Ou alors**

Utiliser le script ci-dessous pour extraire des loci d'intérêt et les concaténer dans un fichier fasta (voir supplementary table 10 de l'article présent dans README pour les position start et end)

In [ ]:
from Bio import SeqIO

# Dossier contenant les fichiers FASTA pour chaque chromosome
fasta_folder = "/home/nguyeho3/Documents/Projet_pangenome/chromosomes/chromosomes_2/"

# Liste des loci à extraire
# Format : {chromosome_filename: [(start1, end1), (start2, end2), ...]}
loci_to_extract = {
    "GWHEQVE00000002.fasta": [(500001, 700000)],
    "GWHEQVJ00000002.fasta": [(500001, 700000)],
    "GWHEQVM00000002.fasta": [(500001,700000)],
    "GWHEQVN00000002.fasta": [(500001,700000)],
    # Ajouter d'autres fichiers et coordonnées si nécessaire
}

# Fichier de sortie
output_file = "extracted_loci_3.fasta"

# Extraire les loci depuis les fichiers FASTA individuels
with open(output_file, "w") as out_file:
    for fasta_file, loci in loci_to_extract.items():
        fasta_path = fasta_folder + fasta_file
        
        try:
            # Charger la séquence du fichier FASTA
            record = next(SeqIO.parse(fasta_path, "fasta"))
            sequence = record.seq
        except FileNotFoundError:
            print(f"Avertissement : Fichier '{fasta_file}' introuvable.")
            continue
        except StopIteration:
            print(f"Avertissement : Fichier '{fasta_file}' est vide.")
            continue
        
        # Extraire les loci pour ce fichier
        for i, (start, end) in enumerate(loci, start=1):
            # Extraire le locus (attention à l'index 0 en Python)
            locus_seq = sequence[start-1:end]
            
            # Créer un header unique pour chaque locus
            header = f">{fasta_file}_locus_{start}_{end}_segment_{i}"
            
            # Sauvegarder dans le fichier FASTA
            out_file.write(f"{header}\n")
            out_file.write(str(locus_seq) + "\n")
            print(f"Locus extrait : {header}")

print(f"Extraction terminée. Résultats sauvegardés dans '{output_file}'.")


Utiliser la commande ci-dessous dans le terminal pour concaténer les séquences chromosomiques d'intérêt en un seul fichier fasta (modifier les noms des fichiers en fonction de la concaténation voulue)

In [ ]:
cat GWHEQVE00000008.fasta GWHEQVJ00000008.fasta GWHEQVM00000008.fasta GWHEQVN00000008.fasta > concatChrom1.fa

**Ou alors**

Vous pouvez utiliser le fichier concatenate.py ci-dessous pour réaliser une concaténation des séquences chromosomiques en fonction du numéro de chromosome:

In [ ]:
from pathlib import Path
from collections import defaultdict

# Répertoire contenant les fichiers .fasta
chemin = Path("Projet_Pangenome/Chromosomes")

# Dictionnaire pour regrouper les fichiers par leur suffixe
groupes_fichiers = defaultdict(list)

# Dictionnaires pour les groupes de contenu
groupes_contenu = {
    "C1": [],
    "C2": [],
    "C3": [],
    "C4": [],
    "C5": [],
    "C6": [],
    "C7": [],
    "C8": [],
    "C9": [],
    "00000010": []
}

# Parcourir les fichiers dans le répertoire
for element in chemin.glob("*.fasta"):
    with open(element, 'r') as chromosome:
        contenu = chromosome.read()
        for key in groupes_contenu.keys():
            if key in contenu:
                groupes_contenu[key].append(contenu)

# Créer le répertoire de sortie s'il n'existe pas
dossier = Path("Projet_Pangenome/Concat")
if not dossier.exists():
    dossier.mkdir(parents=True)

# Écrire les fichiers concaténés
for key, sequences in groupes_contenu.items():
    if sequences:  # Vérifier qu'il y a des séquences à écrire
        output_file = dossier / f"Concat_{key}.fasta"
        with open(output_file, "w") as concat:
            concat.write("\n".join(sequences))

print("Concaténation terminée.")

Créer un index pour chaque fichier fasta concaténé

In [ ]:
samtools faidx Concat_C1.fasta

Utiliser la commande ci-dessous dans le terminal pour utiliser pggb sur la séquence

In [ ]:
nohup pggb -i Concat_C1.fa -n 4 -o Results_PGGB --multiqc

Les résultats seront rangés dans le dossier Results_PGGB (Durée approximative : 3 heures en fonction des capacités de la machine)